# Getting Started
This notebook demonstrates how to use xRHEED to load, inspect, and plot an example RHEED image.

The image used in this tutorial was recorded from a Si(111) surface exhibiting the (7×7) reconstruction. The electron beam was aligned along the $[11\bar{2}]$ crystallographic direction, and the image was captured at room temperature.

## Load xRHEED module
First, import the `xrheed` library along with other useful Python modules.

In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path

import xrheed

## Load a RHEED Image

RHEED images are loaded using dedicated **plugins**.  
For more information on writing custom plugins, see the documentation: *Plugins for Data Loading*.  

The image coordinates - **sx** (horizontal) and **sy** (vertical) - are expressed in **millimeters**.  
This means that the plugin is responsible for converting pixel values to real-world dimensions using the appropriate scaling.


In [ ]:
image_dir = Path("example_data")
image_path = image_dir / "Si_111_7x7_112_phi_00.raw"

rheed_image = xrheed.load_data(image_path, plugin="dsnp_arpes_raw")

## Manual RHEED Data Loading

While writing and using a dedicated plugin is the recommended approach, it is also possible to load RHEED images manually (from **BMP**, **PNG**, **TIFF**, or other supported image formats).  
In this mode, you must provide the necessary calibration parameters directly.  

Use the `load_data_manual()` function as shown below.  
At minimum, the following three parameters are required:

- **`screen_sample_distance`** *(float)* — Distance from sample to screen [mm].  
- **`screen_scale`** *(float)* — Scaling factor [pixels per mm].  
- **`beam_energy`** *(float)* — Beam energy [eV].  

In [ ]:
image_dir = Path("example_data")
image_path = image_dir / "test_rheed.BMP"

rheed_manual = xrheed.load_data_manual(
    image_path,
    screen_sample_distance=350.5,
    screen_scale=9.5,
    beam_energy=18600,
    screen_center_sx_px=749,
    screen_center_sy_px=82,
)

The data are loaded into an `xarray` object, which provides powerful built-in methods for data manipulation and analysis. These methods can be used to inspect, slice, and visualize the RHEED image, as demonstrated below.

In [ ]:
rheed_image.sel(sx=slice(-50, 50), sy=slice(-60, 10)).plot()
plt.show()

## The `.ri` Accessor

Tools dedicated to RHEED images analysis are available through the `.ri` accessor. Please refer to the API documentation to explore all available methods and properties.

For example, you can access basic properties of the RHEED image as shown below.

In [ ]:
print(rheed_image.ri.beta)  # incident angle
print(rheed_image.ri.beam_energy)
print(rheed_image.ri.screen_scale)

## RHEED Image Data Preparation

### Setting the Image Center

Although the plugin used to load the data should provide information about the screen center, it's often necessary to fine-tune the center manually for accurate analysis. The image center is defined as follows:

- **Horizontal center** (`sx = 0`): the line connecting the specular spot and the transmitted beam (not always visible).
- **Vertical center** (`sy = 0`): located at the shadow boundary.

Both values depend on the specific RHEED setup and typically require manual adjustment for each image or image series.

#### Manual Center Adjustment

The image center can be manually adjusted using the method shown below.

First, we plot the suspected center point to visually verify its accuracy.

In [ ]:
center_x = -0.5  # in mm
center_y = 0.5  # in mm

fig, ax = plt.subplots()

rheed_image.ri.plot_image(ax=ax, show_center_lines=True)
ax.plot(center_x, center_y, "b.")

plt.show()

If the position looks correct, we then apply the values using the `apply_image_center` method available through the `.ri` accessor.

> **Note:** Applying this method multiple times will continuously shift the image if `center_x` and `center_y` are non-zero. A recommended usage is to call this function in the same cell that loads the image.


In [ ]:
rheed_image.ri.apply_image_center(center_x=center_x, center_y=center_y)

rheed_image.ri.plot_image(show_center_lines=True)
plt.show()

### Automated Center Search

There are two functions in `xrheed.preparation.alignment` that can be used to find the vertical and horizontal center of a RHEED image.

It is recommended to follow this order:

- First, determine the horizontal center using the value returned by `find_horizontal_center(image)`,
- Then, apply the correction,
- Finally, proceed the same way with `find_vertical_center(image)`.

In [ ]:
from xrheed.preparation.alignment import find_vertical_center, find_horizontal_center

# Try to find the horizontal center of the RHEED image
center_x = find_horizontal_center(rheed_image)
print(center_x)

# Apply the horizontal center correction
rheed_image.ri.apply_image_center(center_x=center_x)

# Find the vertical center, using a specified shadow edge width
center_y = find_vertical_center(rheed_image, shadow_edge_width=5.0)
print(center_y)

# Apply the horizontal center correction
rheed_image.ri.apply_image_center(center_y=center_y)

Alternatively, both functions can be called automatically using the same `apply_image_center` method with the `auto_center=True` attribute.

> **Note:** Automatic center search may not work reliably for asymmetric images. This functionality should be treated as a first approximation and fine-tuned manually afterward, as will be demonstrated.



In [ ]:
# Alternatively, use automatic center detection for both directions
rheed_image.ri.apply_image_center(auto_center=True)

# Plot the adjusted image
rheed_image.ri.plot_image(show_center_lines=True)
plt.show()

### Setting the Incident Angle

The plugin assigns a default value for the incident (grazing) angle, denoted as **β**. However, to use the Ewald construction accurately, this value should be manually set or calculated, as shown below.

If both the mirror-reflected and transmitted spots are visible in the RHEED image, their positions can also be used to fine-tune the shadow edge location (`sy = 0.0`).

Below is an example of how to use the `find_incident_angle()` function provided by the `xrheed.preparation.alignment` module. This function returns the estimated incident angle, and additionally plots useful information such as:

- the shadow edge position (defined precisely between the two spots),
- the distance between the spots,
- and other relevant alignment details.


In [ ]:
from xrheed.preparation.alignment import find_incident_angle

real_beta = find_incident_angle(rheed_image)

To update the image attribute with the real incident angle β, you can use the `.ri` accessor:

In [ ]:
rheed_image.ri.beta = real_beta
rheed_image.ri

## Image Rotation

If the RHEED camera is not perfectly aligned, it may be necessary to rotate the RHEED image to ensure that the shadow edge is perfectly horizontal.


In [ ]:
# Create a copy of an image
rotated_image = rheed_image.copy()

# Rotate the image
rotated_image.ri.rotate(-0.4)

fig, ax = plt.subplots()
rotated_image.ri.plot_image(ax=ax, show_center_lines=True)

# add horizontal line to check the rotation alignment
ax.axhline(-42.0, color="b", linestyle="--")

plt.show()

In [ ]:
# Use now the rotated image for further analysis
rheed_image = rotated_image

# Apply automatic center search again after rotation
rheed_image.ri.apply_image_center(auto_center=True)

## Screen ROI

Define the region of interest (ROI) for the RHEED image. This ROI is used to set the limits for the x and y axes in the `plot_image()` method.

The example below also demonstrates the use of the `auto_levels` argument, which automatically sets the `vmin` and `vmax` values for image contrast adjustment.


In [ ]:
rheed_image.ri.screen_roi_width = 60
rheed_image.ri.screen_roi_height = 60

# Use automatic levels adjustment
# auto_levels: percentage of pixels to clip at both low and high ends.
rheed_image.ri.plot_image(auto_levels=2.0)
plt.show()

## High-Pass Filter

Apply a high-pass filter to remove the homogeneous background signal from the RHEED image.

You can adjust the following parameters:

- `sigma` (in mm): controls the width of the Gaussian kernel used for blurring,
- `threshold`: scales the blurred image before subtraction.

The filtered image is computed as:

`filtered_image = real_image - threshold * blurred_image`


In [ ]:
from xrheed.preparation.filters import high_pass_filter

sigma = 3.0
threshold = 0.8

hp_rheed_image = high_pass_filter(rheed_image, sigma=sigma, threshold=threshold)

hp_rheed_image.ri.plot_image(auto_levels=1.0)
plt.show()